In [ ]:
#######################################################
#  THE IPYNB FILE IS USED IN THE UII TO PRECESS DATA  #
#                                                     #
#######################################################

In [ ]:
# 1.先把 mask 盖在图像上然后把图像拿出来看一看

import torchio as tio
import numpy as np
import pandas as pd
import os
import csv
import torch
import glob
from tqdm import tqdm
import numpy as np
from scipy.ndimage import binary_dilation
import csv



def dilate_mask(mask, iterations=20):
    # 创建一个结构元素
    # struct = np.ones((10, 10, 10))
    # 使用binary_dilation函数进行膨胀
    dilated_mask = binary_dilation(mask, iterations=iterations)
    return dilated_mask


def read_names_from_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        names = []
        for row in reader:
            # print(row)
            name = row[0]
            names.append(name)
    return names

def extract_subvolume(arr):
    # 找到非零元素的索引
    nonzero_indices = np.transpose(np.nonzero(arr))
    
    if len(nonzero_indices) == 0:
        return None, None  # 如果没有非零元素，则返回None
    
    # 获取包含所有非零元素的最小长方体的边界
    min_indices = np.min(nonzero_indices, axis=0)
    max_indices = np.max(nonzero_indices, axis=0) + 1
    
    # 提取最小长方体
    subvolume = arr[min_indices[0]:max_indices[0], min_indices[1]:max_indices[1], min_indices[2]:max_indices[2]]
    
    return subvolume, (min_indices, max_indices)

def extract_subarray(arr):
    # 找到所有大于0的元素的坐标
    coords = np.argwhere(arr > 0)
    # 计算最小和最大的坐标
    min_coords = coords.min(axis=0)
    max_coords = coords.max(axis=0)
    # 提取子数组
    subarray = arr[min_coords[0]:max_coords[0]+1, min_coords[1]:max_coords[1]+1, min_coords[2]:max_coords[2]+1]
    return subarray, min_coords, max_coords


def transform_coords(coords, min_coords):
    # 计算新坐标
    new_coords = []
    for c in coords:
        c[0:3] = c[0:3] - min_coords
        # c[3:6] = c[3:6] - min_coords
        if min(c[0:3]) < 0:
            print(min(c[0:3]))
            print('ERROR!  the bbox in out of the mask aera!')
        else:
            new_coords.append(c)
        
    return new_coords




def write_to_csv(filename, name, coords):
    try:
        # 打开文件以追加模式，如果文件不存在则创建
        with open(filename, 'a', newline='') as file:
            writer = csv.writer(file)
            
            # 写入列表的内容
            for coord in coords:
                data_list = [name, coord[0], coord[1], coord[2], coord[3], coord[4], coord[5]]
                writer.writerow(data_list)
        
        # print(f"Data written to {filename} successfully.")
    except Exception as e:
        print(f"Error writing to {filename}: {str(e)}")


def process_nii_image(image_path, bboxes, output_path):
    # 加载nii图像
    img = tio.ScalarImage(image_path)
    data = img.data[0, :, :, :]
    # print(bboxes)
    # 遍历每个框
    for bbox in bboxes:
        
        x1, y1, z1, x2, y2, z2 = map(int, bbox)
        # print(x1, y1, z1, x2, y2, z2)
        # 将框内的像素值设置为50
        # 将框的边线上的像素值设置为50
        data[x1:x2+1, y1:y1+1, z1:z2+1] = 10
        data[x1:x2+1, y2:y2+1, z1:z2+1] = 10
        data[x1:x1+1, y1:y2+1, z1:z2+1] = 10
        data[x2:x2+1, y1:y2+1, z1:z2+1] = 10
        data[x1:x2+1, y1:y2+1, z1:z1+1] = 10
        data[x1:x2+1, y1:y2+1, z2:z2+1] = 10
        # print(f'data max is {data.max()}')
    
    # 保存结果
    affine =  np.diag([-0.7, -0.7, 0.7, 1])
    new_img = tio.ScalarImage(tensor=data.unsqueeze(0), affine=affine)
    new_img.save(output_path)


def convert_coordinates(coords): 
    x1y1z1 = []
    for coord in coords:
        # print(coord)
        x, y, z, w, h, d = coord
        x1 = x - w / 2. 
        y1 = y - h / 2. 
        z1 = z - d / 2. 
        x2 = x + w / 2. 
        y2 = y + h / 2. 
        z2 = z + d / 2. 
        x1y1z1.append([x1, y1, z1, x2, y2, z2])

    return x1y1z1



#* 先盖 mask
parts = ['testing', 'training', 'validation']
for part in parts:
    csv_path = f'/data/julia/data_lymph/csv_file/part_{part}_names.csv'
    names = read_names_from_csv(file_path=f'/data/julia/data_lymph/csv_file/{part}_names.csv')
    csv_filename = f'/data/julia/data_lymph/anno/{part}_refine.csv'  # 文件名
    # names = ['01190830220138']
    test = 0
    for name in tqdm(names):
        if os.path.exists(f'/data/julia/data_lymph/{part}_processed/{name}.nii.gz'):
            print(f"File {name}.nii.gz exists. Skipping this iteration.")
            continue
        # print(f"Processing {file_name}...")
        img_path = f'/data/julia/data_lymph/{part}/{name}/'
        file_name = os.listdir(img_path)
        img = tio.ScalarImage(os.path.join(img_path, file_name[0]))

        # * 窗宽窗位设置一下
        clamped = tio.Clamp(out_min=-160., out_max=240.)
        clamped_img = clamped(img)
        # clamped_img.save(f'/data/julia/data_lymph/{part}_processed/{name}_clamp.nii.gz')

        # * resample到（0.7， 0.7， 0.7）
        resample = tio.Resample(0.7)
        clamped_img = resample(clamped_img)
        # print(clamped_img.spacing)
        
        # * 归一化到 0-1 之间
        data_max = clamped_img.data.max()
        data_min = clamped_img.data.min()
        norm_data = (clamped_img.data - data_min) / (data_max - data_min)
        # norm_img = tio.ScalarImage(tensor=norm_data, affine=clamped_img.affine)
        # norm_img.save(f'/data/julia/data_lymph/{part}_processed/{name}_norm.nii.gz')

        mask_path = f'/data/julia/data_lymph/{part}_mask/{name}/mediastinum.nii.gz'
        mask = tio.LabelMap(mask_path)
        mask = resample(mask)

        # #* 盖上 mask
        dilated_mask = dilate_mask(mask.data[0, :, :, :])
        img_mask = norm_data * torch.tensor(dilated_mask).unsqueeze(0)
        # img_mask = tio.ScalarImage(tensor=img_mask, affine=clamped_img.affine)
        # img_mask.save(f'/data/julia/data_lymph/{part}_processed/{name}_dilatepro.nii.gz')

        #! 到现在是一整张图像被处理结束， 然后下面开始处理想对应的坐标
    
        # * 这样的话先处理一下这个坐标吧

        #* 读取csv文件中的世界坐标
        # name = '01190830220138'
        worldcoord = pd.read_csv(f'/data/julia/data_lymph/lymph_csv_refine/CTA_thin_std_{part}_lymph_refine.csv')
        raw = worldcoord[worldcoord['image_path'].str.contains(name)]

        coords = []
        for i in range(len(raw)):
            x = raw.iloc[i, 2]
            y = raw.iloc[i, 3]
            z = raw.iloc[i, 4]
            width = raw.iloc[i, 5]
            height = raw.iloc[i, 6]
            depth = raw.iloc[i, 7]
            coords.append([x, y, z, width, height, depth]) # 这个是世界坐标系
        # print(coords)

        # * 把世界坐标系转化为图像坐标系
        # img = tio.ScalarImage('/data/julia/data_lymph/testing/01190830220138/std_1.0_6.nii.gz')
        origin = img.origin
        # print(f'the origin is {origin}')
        img_coords = []
        for coord in coords:
            img_coord = (np.array(coord[0:3]) - np.array(origin) * np.array([-1., -1., 1.]) ) / np.array([0.7, 0.7, 0.7]) # img.spacing
            coord[3: 6] = coord[3: 6] / np.array([0.7, 0.7, 0.7])
            img_coords.append([img_coord[0], img_coord[1], img_coord[2], coord[3], coord[4], coord[5]])   #! xyzwhd


        # 现在开始扣数据
        #* 把数据分割出来，旁边的都是0的东西给去掉
        # nii_img = tio.ScalarImage('/data/julia/data_lymph/testing_processed/01190830220138_dilatepro.nii.gz')
        crop_img, coord = extract_subvolume(np.array(img_mask[0, :, :, :]))
        crop_img = tio.ScalarImage(tensor=torch.tensor(crop_img).unsqueeze(0), affine=np.diag([-0.7, -0.7, 0.7, 1]))
        crop_img.save(f'/data/julia/data_lymph/{part}_processed/{name}.nii.gz')
        # print(coord)

        new_coords = transform_coords(coords=img_coords, min_coords=coord[0])
        # print(name)
        # print(new_coords)

        # 调用函数来写入数据
        write_to_csv(csv_filename, name, new_coords)

        if test == 0:
            xyz_coords = convert_coordinates(img_coords)
            process_nii_image(f'/data/julia/data_lymph/{part}_processed/{name}.nii.gz', xyz_coords, f'/data/julia/data_lymph/{part}_processed/{name}_testbbox.nii.gz')
            test += 1

    

In [ ]:
import torchio as tio
import numpy as np
import pandas as pd
import os
import csv
import torch
import glob
from tqdm import tqdm
import numpy as np
from scipy.ndimage import binary_dilation
import csv
from scipy.ndimage import gaussian_filter



def read_imgcoord_fromcsv(name, part):
    #* 读取csv文件中的世界坐标
    # name = '01190830220138'
    imgcoord = pd.read_csv(f'/data/julia/data_lymph/anno/{part}_refine.csv')
    raw = imgcoord[imgcoord['name']=="'" + name]
    coords = []
    for i in range(len(raw)):
        x = raw.iloc[i, 1]
        y = raw.iloc[i, 2]
        z = raw.iloc[i, 3]
        width = raw.iloc[i, 4]
        height = raw.iloc[i, 5]
        depth = raw.iloc[i, 6]
        coords.append([x, y, z, width, height, depth]) # 这个是图像坐标系

    return coords


def create_hmap_v4(coordinates, shape):
    arr = np.zeros(shape)
    for coords in coordinates:
        coord = [int(x) for x in coords[0:3]]
        whd = [int(x) for x in coords[3:6]]
        kernel = create_gaussian_kernel_v4(whd)
        arr = place_gaussian(arr, kernel, coord)

    return arr


def create_gaussian_kernel_v4(whd):
    size_max = int(np.max(whd))
    size_min = int(np.min(whd))
    size_mid = int(sorted(whd)[1])

    array_large = create_gaussian_base(size_max, 0.5)
    array_small = create_gaussian_base(size_min, 0.5)
    array_midum = create_gaussian_base(size_mid, 0.5)

    combined_kernel = combine_gaussian_kernels(array_large, array_small, array_midum)

    return combined_kernel


def create_gaussian_base(size, threshold):

    if size <= 9:
        _size = 9
        half_dis = (_size + 1) / 2.
    else:
        _size = size
        if _size % 2 != 1:  # 如果size是偶数就变成奇数
            half_dis = _size / 2.
            _size = _size + 1
        else:
            half_dis = (_size + 1) / 2.

    if threshold == 0.5:
        sigma = np.sqrt(half_dis**2 / (2 * np.log(2)))
    elif threshold == 0.8:
        sigma = np.sqrt(half_dis**2 / (2 * (np.log(5) - np.log(4))))
    elif threshold == 0.3:
        sigma = np.sqrt(half_dis**2 / (2 * (np.log(10) - np.log(3))))
    else:
        print(f'when x = distance, the y wrong input, now the threshold is {threshold}')

    kernel = np.zeros((int(_size), int(_size), int(_size)))
    center = tuple(s // 2 for s in (int(_size), int(_size), int(_size)))
    kernel[center] = 1
    gassian_kernel = gaussian_filter(kernel, sigma=sigma)

    arr_min = gassian_kernel.min()
    arr_max = gassian_kernel.max()
    normalized_arr = (gassian_kernel - arr_min) / (arr_max - arr_min) # 归一化到 0-1 之间
    # print(f'in the create_gaussian_base , the max is {normalized_arr.max()}, the min is {normalized_arr.min()}')
    return normalized_arr


def combine_gaussian_kernels(kernel_large, kernel_small, kernel_midum):
    center_large = np.array(kernel_large.shape) // 2
    small_shape = np.array(kernel_small.shape[0]) // 2
    midum_shape = np.array(kernel_midum.shape[0]) // 2

    kernel_large[center_large[0] - small_shape : center_large[0] + small_shape + 1, 
                 center_large[1] - small_shape : center_large[1] + small_shape + 1, 
                 center_large[2] - small_shape : center_large[2] + small_shape + 1, ] += kernel_small[:, :, :]
    
    kernel_large[center_large[0] - midum_shape : center_large[0] + midum_shape + 1, 
                 center_large[1] - midum_shape : center_large[1] + midum_shape + 1, 
                 center_large[2] - midum_shape : center_large[2] + midum_shape + 1, ] += kernel_midum[:, :, :]
    
    arr_min = kernel_large.min()
    arr_max = kernel_large.max()
    normalized_arr = (kernel_large - arr_min) / (arr_max - arr_min) # 归一化到 0-1 之间
    # print(f'in the combine_gaussian_kernels , the max is {normalized_arr.max()}, the min is {normalized_arr.min()}')
    return normalized_arr


def place_gaussian(arr, kernel, pos):
    x, y, z = pos
    kx, ky, kz = kernel.shape
    # 计算高斯核在数组中的位置
    x1, x2 = max(0, x-kx//2), min(arr.shape[0], x+kx//2+1)
    y1, y2 = max(0, y-ky//2), min(arr.shape[1], y+ky//2+1)
    z1, z2 = max(0, z-kz//2), min(arr.shape[2], z+kz//2+1)
    # 计算高斯核在自身中的位置
    kx1, kx2 = max(0, kx//2-x), min(kx, kx//2-x+arr.shape[0])
    ky1, ky2 = max(0, ky//2-y), min(ky, ky//2-y+arr.shape[1])
    kz1, kz2 = max(0, kz//2-z), min(kz, kz//2-z+arr.shape[2])
    # 将高斯核放置在指定位置
    arr[x1:x2,y1:y2,z1:z2] = np.maximum(arr[x1:x2,y1:y2,z1:z2], kernel[kx1:kx2,ky1:ky2,kz1:kz2])

    return arr


nii_path = '/data/julia/data_lymph/testing_processed/202004290225_testbbox.nii.gz'
name = '202004290225'
imgcoords = read_imgcoord_fromcsv(name, part='testing')
# print(imgcoords)
nii_img = tio.ScalarImage(nii_path)
shape = nii_img.shape
print(shape)


hmap = create_hmap_v4(imgcoords, shape[1:4])
hmap_nii = tio.ScalarImage(tensor=torch.tensor(hmap).unsqueeze(0), affine=nii_img.affine)
hmap_nii.save('/data/julia/data_lymph/hmap.nii')